# Tutorial 3 - Sediment Routing

The latest HydroCNHS can also routing the total suspended sediment. Here, we demostrate how to construct a model that has sediment routing feature activated. We adopted a subbasin, Sb5, in Susquehena River Basin. For more details, please refer to (Lin et al., 2023).

Lin, C.-Y., Yang, Y. E., & Chaudhary, A. K. (2023). Pay-for-practice or Pay-for-performance? A coupled agent-based evaluation tool for assessing sediment management incentive policies. Journal of Hydrology, 624, 129959.

<img src="./NB_Figs/SRB_map.jpg" alt="SRB_map" width="600"/>

## Create a draft `model.yaml` using model builder

In [2]:
import os
import pandas as pd
import hydrocnhs

# Set the working directory
wd = os.path.abspath(
    os.path.join(
        os.path.dirname(hydrocnhs.__file__),
        '..', '..', 'tutorials',
        'Tutorial_3-Sediment_Routing'
        )
    )
data_path = os.path.join(wd, "Data")

# We only model a subbasin (Sb5) in this tutorial
sb = "Sb5"

# Load data
df_hu8 = pd.read_csv(os.path.join(data_path, "SRB_HU8.csv"))
df_hu8.index = list(df_hu8.ID)
df_hu8_sb = df_hu8.loc[[x for x in df_hu8["ID"] if sb in x], :]

df_agt = pd.read_csv(os.path.join(data_path, "agt_shp_info.csv"))

# Create a model using model builder
mb = hydrocnhs.ModelBuilder(wd)
mb.set_water_system("1984/01/01", "2020/12/31")
mb.set_rainfall_runoff(
    list(df_hu8_sb["ID"]),
    list(df_hu8_sb["area_ha"]),
    list(df_hu8_sb["central_la"]),
    runoff_model="GWLF"
    )

mb.set_routing_outlet(
    routing_outlet="Sb5_3",
    upstream_outlet_list=["Sb5_1", "Sb5_2", "Sb5_3"],
    flow_length_list=[113610, 113610, 0] # km
    )

# Activate sediment routing section in the model
mb.set_sediment(start_month=4)

outlets = mb.model["WaterSystem"]["Outlets"]
for o in outlets:
    mb.add_sediment(
        subbasin=o,
        area_list=list(df_agt.loc[df_agt["ID"]==o, "area_ha"]),
        cool_months=[10,11,12,1,2,3],
        K_list=list(df_agt.loc[df_agt["ID"]==o, "Value_K"]),
        Ac=0.12, Aw=0.3,
        LS_list=list(df_agt.loc[df_agt["ID"]==o, "Value_LS"]),
        DR=float(df_hu8.loc[o, "DR"]),
        id_list=list(df_agt.loc[df_agt["ID"]==o, "Agt_ID"])
        )

mb.write_model_to_yaml(filename="model.yaml")

Follow the following steps to create model & ABM script templates:
	Step 1: set_water_system()
	Step 2: set_rainfall_runoff()
	Step 3: set_routing_outlet(), one at a time.
	Step 4: (optional) set_sediment().
	Step 5: (optional) add_sediment(), one at a time.
	Step 6: (optional) set_ABM().
	Step 7: (optional) add_agent().
	Step 8: (optional) add_institution().
	Step 9: write_model_to_yaml()
	Step 10: (optional) gen_ABM_script_template()
Open the generated draft 'model.yaml' (& ABM module template) and further edit them.
Use .help to re-print the above instructions.
Model configuration file (.yaml) have been save at C:\Users\CL\Documents\GitHub\HydroCNHS\tutorials\Tutorial_3-Sediment_Routing\model.yaml. Please open the file and further edit it.


We encourage you to open `model.yaml` to give a sense of how it looks like. Users can follow the calibration steps shown in the previous tutorials to calibrate the model. Here we will skip the calibration step and adopted the calibrated parameters used in (Lin et al., 2023).

The calibrated model is located in `./Calibrated_model/Best_hydro_sed_Sb5_seed3_iter100.yaml`.

Now, let's used the calibrated model to run a simulation.

## [Pending for completion] Run sediment routing simulation 

In [9]:
import pickle

# Load data
inputs_path = os.path.join(wd, "Inputs")

with open(os.path.join(inputs_path, "hydro_cali_1984_2020.pkl"), "rb") as file:
    (prec, temp, pet, Q_M, Q_Y, sed_M, sed_Y, _) = pickle.load(file)

with open(os.path.join(inputs_path, "cli_sed_sim_1999_2020.pkl"), "rb") as file:
    prec_sed = pickle.load(file)[0]
    

In [10]:
# Load model.yaml
model_dict = hydrocnhs.load_model(
    os.path.join(wd, "Calibrated_model", "Best_hydro_sed_Sb5_seed3_iter100.yaml")
    )
# Change the working directory
model_dict["Path"]["WD"] = wd

# Run the model
model = hydrocnhs.Model(model_dict)
Q = model.run(temp, prec, pet)


13515it [00:00, 20758.76it/s]

In [ ]:
        # Convert 1D array to a list of dataframes.
        df_list = cali.Convertor.to_df_list(individual, formatter)
        # Feed dataframes in df_list to model dictionary.
        model_dict2 = deepcopy(model_dict)
        for i, df in enumerate(df_list):
            s = df_name[i].split("_")[0]
            model_dict2 = hy.load_df_to_model_dict(model_dict2, df, s, "Pars")

        ##### Run simuluation
        #model = r"C:\Users\Philip\OneDrive\Lehigh\0_Proj4_SRB_WQ\Model\exp1_hydro_hu8\Cali_gwlf_abm_KGE_5\Best_gwlf_abm_KGE.yaml"
        model = hy.Model(model_dict2, name)
        Q = model.run(temp, prec, pet)

        ##### Get simulation data
        # Streamflow of routing outlets.
        cali_target = ['Sb1_2', 'Sb2_3', 'Sb3_2', 'Sb4_6', 'Sb5_3', 'Sb6_2']
        cali_target = [ca for ca in cali_target if sb in ca]
        cali_period = ("1985-1-1", "2011-12-31")
        vali_period = ("2012-1-1", "2020-12-31")
        sim_Q_D = pd.DataFrame(Q, index=model.pd_date_index)[cali_target]
        # Resample the daily simulation output to monthly and annually outputs.
        sim_Q_M = sim_Q_D[cali_target].resample("MS").mean()
        sim_Q_Y = sim_Q_D[cali_target].resample("YS").mean()

        df_cali_Q_M = cal_batch_indicator(cali_period, cali_target, Q_M, sim_Q_M)
        df_cali_Q_Y = cal_batch_indicator(cali_period, cali_target, Q_Y, sim_Q_Y)

        df_vali_Q_M = cal_batch_indicator(vali_period, cali_target, Q_M, sim_Q_M)
        df_vali_Q_Y = cal_batch_indicator(vali_period, cali_target, Q_Y, sim_Q_Y)

        ##### Run sediment simulation
        run_sed_TF = False
        if isinstance(current_generation, str):
            if current_generation == "best":
                run_sed_TF = True
        else:
            if current_generation >= 5:
                run_sed_TF = True
        if run_sed_TF:
            sed_sim_period = ("1999-1-1", "2019-12-31") # -1 and +1 year
            sed_perf_period = ("2000-1-1", "2019-12-31")
            dc = model.dc
            Q_frac = dc.get_field("Q_frac", copy=True)
            Q_frac = {ro: pd.DataFrame(sbs, index=model.pd_date_index)[sed_sim_period[0]:"2020-12-31"] \
                      for ro, sbs in Q_frac.items()}
            sim_seq = model.sys_parsed_data["SimSeq"]

            prec_sed = cli_sed_sim_data[0]
            _, _, sim_sed_M, sim_sed_Y = sim_sed(
                model_dict2, prec_sed, Q_frac, sed_sim_period[0], sed_sim_period[1], sim_seq)

            df_perf_sed_M = cal_batch_indicator(sed_perf_period, cali_target, sed_M, sim_sed_M)
            df_perf_sed_Y = cal_batch_indicator(sed_perf_period, cali_target, sed_Y, sim_sed_Y)

In [11]:

# Run sediment simulation
sed_sim_period = ("1999-1-1", "2019-12-31") # -1 and +1 year
sed_perf_period = ("2000-1-1", "2019-12-31")
dc = model.dc
Q_frac = dc.get_field("Q_frac", copy=True)
Q_frac = {ro: pd.DataFrame(sbs, index=model.pd_date_index)[sed_sim_period[0]:"2020-12-31"] \
            for ro, sbs in Q_frac.items()}
sim_seq = model.sys_parsed_data["SimSeq"]

_, _, sim_sed_M, sim_sed_Y = sim_sed(
    model_dict, prec_sed, Q_frac, sed_sim_period[0], sed_sim_period[1], sim_seq)

df_perf_sed_M = cal_batch_indicator(sed_perf_period, cali_target, sed_M, sim_sed_M)
df_perf_sed_Y = cal_batch_indicator(sed_perf_period, cali_target, sed_Y, sim_sed_Y)
fitness = (df_cali_Q_M.loc["Mean", "KGE"] + df_perf_sed_M.loc["Mean", "KGE"])/2
##### Record performance
performance[f] = [df_cali_Q_M.loc["Mean", "KGE"], df_perf_sed_M.loc["Mean", "KGE"],
                    df_cali_Q_Y.loc["Mean", "KGE"], df_perf_sed_Y.loc["Mean", "KGE"],
                    fitness]
sim_sed_M_dict[f] = sim_sed_M[sed_perf_period[0]: sed_perf_period[1]]
sim_sed_Y_dict[f] = sim_sed_Y[sed_perf_period[0]: sed_perf_period[1]]

NameError: name 'sim_sed' is not defined

In [ ]:
Q

{'Sb5_1': array([400.40242073, 171.15351551,  76.21969939, ...,  61.01088311,
         28.18771103,  13.18862614]),
 'Sb5_2': array([505.02861438, 307.47435197, 200.20797688, ...,  66.26275449,
         60.26508671,  30.73452566]),
 'Sb5_3': array([1.26208804e-02, 3.34247654e-01, 1.99366605e+00, ...,
        1.05574546e+02, 1.52564172e+02, 2.25909361e+02])}

In [ ]:
##### Get simulation data
sim_Q_D = pd.DataFrame(Q, index=model.pd_date_index)[['Sb5_3']]
# Resample the daily simulation output to monthly and annually outputs.
sim_Q_M = sim_Q_D.resample("MS").mean()
sim_Q_Y = sim_Q_D.resample("YS").mean()

sim_Q_M.head()
sed_M.head()

,Sb1_2,Sb2_3,Sb3_2,Sb4_6,Sb5_3,Sb6_2
2000-01-01,NaN,NaN,22638.77672,4288.71236,1387.855442,53292.52408
2000-02-01,NaN,NaN,199943.35360,64586.96488,6708.625680,230197.94000
2000-03-01,NaN,NaN,285445.44560,58917.06488,21373.255040,492147.32000
2000-04-01,NaN,NaN,190508.64000,60146.29920,14968.536000,372852.62400
2000-05-01,NaN,NaN,99554.37216,15467.48720,9308.615024,244667.52480
